In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, Callback
import keras

keras.backend.set_image_data_format('channels_last')

# telegram
import telegram

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time

Using TensorFlow backend.


In [2]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        #tensor =  tf.convert_to_tensor(self.settings_str)
        #summary = tf.summary.text("Run_Settings", tensor)
        #
        #with  tf.Session() as sess:
        #    s = sess.run(summary)
        #    self.writer.add_summary(s)

In [3]:
class TelegramCallback(Callback):

    def __init__(self, config):
        super(TelegramCallback, self).__init__()
        self.user_id = config['telegram_id']
        self.model_name = config['model_name']
        self.bot = telegram.Bot(config['token'])

    def send_message(self, text):
        try:
            self.bot.send_message(chat_id=self.user_id, text=text)
        except Exception as e:
            print('Message did not send. Error: {}.'.format(e))

    def on_train_begin(self, logs={}):
        text = 'Start training model {} | {}'.format(self.model.name, self.model_name)
        self.send_message(text)

    def on_epoch_end(self, epoch, logs={}):
        text = '{} | Epoch {}.\n'.format(self.model_name, epoch)
        for k, v in logs.items():
            text += '{}: {:.4f}; '.format(k, v)
        self.send_message(text)

In [4]:
smartphone = 'S3Mini'
if smartphone == "N5X":
    pixels = {"width": 1080, "height": 1920}
    constant_pixels = 0.06
elif smartphone == "S3Mini":
    pixels = {"width": 480, "height": 800}
    constant_pixels = 0.1089
elif smartphone == "S4":
    pixels = {"width": 1080, "height": 1920}
    constant_pixels = 0.0577
elif smartphone == "N6":
    pixels = {"width": 1440, "height": 2560}
    constant_pixels = 0.05109

In [5]:
df = pd.read_pickle("../../data/step03.pkl")
df = df[(df.Phone == smartphone) ]
df.Sensor = df.Sensor.apply(lambda x: x[-57:-11])
df.Sensor = df.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

np.random.seed(42)
lst = df.PId.unique()
np.random.shuffle(lst)

shape = df.Sensor.iloc[0].shape

dfTrain = df[df.PId.isin(lst[:14])]
dfTest = df[df.PId.isin(lst[14:])]

train_x = np.concatenate(dfTrain.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
train_y = dfTrain[['XPress', 'YPress']].values
train_x = np.concatenate([train_x,train_x,train_x,train_x,train_x,train_x,train_x,train_x])
train_y = np.concatenate([train_y,train_y,train_y,train_y,train_y,train_y,train_y,train_y])

test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
test_y = dfTest[['XPress', 'YPress']].values

In [6]:
shape

(46, 6, 1)

In [ ]:
with tf.device('/device:GPU:0'):
    config = tf.ConfigProto(log_device_placement = True, allow_soft_placement = True)
    config.gpu_options.allow_growth=True
    config.gpu_options.per_process_gpu_memory_fraction = 0.2
    config.gpu_options.allocator_type = 'BFC'
    
    with tf.Session(config=config):
        tf.get_default_graph()
        model = Sequential()

        model.add(Conv2D(filters = 32, kernel_size = (5,1),padding = 'Same', 
                         activation ='relu', activity_regularizer=keras.regularizers.l2(0.001), input_shape = (shape)))
        model.add(BatchNormalization(axis=-1))        
        model.add(MaxPool2D(pool_size=(2,1), padding='Same'))
        model.add(Dropout(0.5))
        
        model.add(Conv2D(filters = 16, kernel_size = (5,1),padding = 'Same', activation ='relu', activity_regularizer=keras.regularizers.l2(0.001), input_shape = (shape)))
        model.add(BatchNormalization(axis=-1))        
        model.add(MaxPool2D(pool_size=(2,1), padding='Same'))
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        model.add(Dense(128, activity_regularizer=keras.regularizers.l2(0.001)))
        model.add(keras.layers.advanced_activations.ReLU())
        model.add(Dropout(0.5))
        
        model.add(Dense(32, activity_regularizer=keras.regularizers.l2(0.001)))
        model.add(keras.layers.advanced_activations.ReLU())
        model.add(Dense(2))
        
        model.add(Dense(8, activity_regularizer=keras.regularizers.l2(0.001)))
        model.add(keras.layers.advanced_activations.ReLU())
        model.add(Dense(2))
        
        # ATTENTION learning rate will be reset!
        #model.load(str(Path.home()) + "/data/models/" + "IMU_N5X_20190115_004501")

        ## Compiling the model to generate a model
        def eucInMM(y_true, y_pred):
            return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
        def euc(y_true, y_pred):
            return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))#
        
        def eucStdMM(y_true, y_pred):
            return K.std(K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1))) * constant_pixels
        def rmse(y_true, y_pred):
            return K.sqrt(K.mean(K.square(y_true - y_pred)))

        optimizer = optimizers.Adam()
        #optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
        model.compile(loss="mse", optimizer=optimizer, metrics=[eucInMM, eucStdMM])

        # Broadcast progress to the tensorboard.
        readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
        file_name = "IMU_" + smartphone + "_" + readable_timestamp
        tensorboardFolder = "/srv/share/tensorboardfiles/" + file_name

        config = ""
        for layer in model.layers:
            config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
        #config += "Batch Size: " + str(batch_size) + "\n\n"
        #config += "Num of Epochs: " + str(epochs) + "\n\n"

        print(file_name)
        model.summary()

        callbacks = [
            ModelCheckpoint(str(Path.home()) + "/data/models/"+ file_name + ".h5",
                            monitor='val_eucInMM',
                            verbose=0,
                            save_best_only=False,
                            save_weights_only=False
                           ),
            LoggingTensorBoard(settings_str_to_log=config,
                               log_dir=tensorboardFolder,
                               histogram_freq=0,
                               write_graph=True,
                               write_images=True,
                               update_freq='epoch'
                              ),
            ReduceLROnPlateau(monitor='val_loss',
                              patience=10,
                              verbose=1,
                              factor=0.9,
                              min_lr=0.00001),
            TelegramCallback(
                {
                    'token': "707020808:AAHJ22G0diUExRP4JUITk1VKuPtBPUVQ-J4",
                    'telegram_id': -320890356,
                    'model_name': file_name,
                }
            )
        ]

        #history = model.fit_generator(myGenerator("train", batch_size),
        #                              steps_per_epoch=len(train_x) // batch_size,
        #                              epochs=1000,
        #                              verbose=1,
        #                              validation_data=myGenerator("test", batch_size),
        #                              validation_steps=len(test_x) // batch_size,
        #                              callbacks=callbacks
        #                             )
        
        history = model.fit(train_x, train_y,
                           batch_size=100,
                           epochs=10000,
                           verbose=2,
                           validation_data=(test_x, test_y),
                           callbacks=callbacks)
        score = model.evaluate(np.expand_dims(test_x,1),np.expand_dims(test_y,1),verbose=2)
        print('%s: %.2f' % (model.metrics_names[1], score[1]))
        print('Baseline ConvLSTM Error: %.2f' %(1 - score[1]))

IMU_S3Mini_20190205_001946
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 6, 32)         192       
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 6, 32)         128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 6, 32)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 6, 32)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 6, 16)         2576      
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 6, 16)         64        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 6, 16)   

Epoch 38/10000
 - 11s - loss: 6154.2328 - eucInMM: 10.1744 - eucStdMM: 5.6972 - val_loss: 9660.5765 - val_eucInMM: 12.6804 - val_eucStdMM: 7.6604
Epoch 39/10000
 - 11s - loss: 6085.9486 - eucInMM: 10.1143 - eucStdMM: 5.6751 - val_loss: 8891.5299 - val_eucInMM: 12.0624 - val_eucStdMM: 7.4235
Epoch 40/10000
 - 11s - loss: 6063.2876 - eucInMM: 10.0983 - eucStdMM: 5.6676 - val_loss: 8929.2823 - val_eucInMM: 12.1591 - val_eucStdMM: 7.3759
Epoch 41/10000
 - 11s - loss: 5955.8380 - eucInMM: 10.0272 - eucStdMM: 5.5901 - val_loss: 8920.7054 - val_eucInMM: 12.1092 - val_eucStdMM: 7.4276
Epoch 42/10000
 - 11s - loss: 5967.1243 - eucInMM: 10.0265 - eucStdMM: 5.6322 - val_loss: 9106.1702 - val_eucInMM: 12.3203 - val_eucStdMM: 7.4321
Epoch 43/10000
 - 11s - loss: 5939.6632 - eucInMM: 10.0022 - eucStdMM: 5.6258 - val_loss: 8855.9224 - val_eucInMM: 12.0906 - val_eucStdMM: 7.3992
Epoch 44/10000
 - 11s - loss: 5937.6331 - eucInMM: 10.0014 - eucStdMM: 5.6400 - val_loss: 8913.0957 - val_eucInMM: 12.1524 -

Epoch 101/10000
 - 11s - loss: 5020.9620 - eucInMM: 9.2546 - eucStdMM: 5.2598 - val_loss: 8972.2630 - val_eucInMM: 12.2105 - val_eucStdMM: 7.5865
Epoch 102/10000
 - 11s - loss: 5017.9158 - eucInMM: 9.2277 - eucStdMM: 5.2989 - val_loss: 8870.5411 - val_eucInMM: 12.1196 - val_eucStdMM: 7.5543
Epoch 103/10000
 - 11s - loss: 5075.9936 - eucInMM: 9.3129 - eucStdMM: 5.2855 - val_loss: 9078.4061 - val_eucInMM: 12.3062 - val_eucStdMM: 7.5980
Epoch 104/10000
 - 11s - loss: 5002.5009 - eucInMM: 9.2388 - eucStdMM: 5.2543 - val_loss: 8803.3904 - val_eucInMM: 12.0779 - val_eucStdMM: 7.5326
Epoch 105/10000
 - 11s - loss: 4974.4096 - eucInMM: 9.2046 - eucStdMM: 5.2568 - val_loss: 8942.8079 - val_eucInMM: 12.2313 - val_eucStdMM: 7.5059
Epoch 106/10000
 - 11s - loss: 4992.7429 - eucInMM: 9.2062 - eucStdMM: 5.2972 - val_loss: 8928.1552 - val_eucInMM: 12.1565 - val_eucStdMM: 7.5753
Epoch 107/10000
 - 10s - loss: 5002.7721 - eucInMM: 9.2303 - eucStdMM: 5.2735 - val_loss: 8968.7540 - val_eucInMM: 12.1729 -

Epoch 156/10000
 - 10s - loss: 4755.0199 - eucInMM: 9.0028 - eucStdMM: 5.1857 - val_loss: 9087.8069 - val_eucInMM: 12.3356 - val_eucStdMM: 7.6119
Epoch 157/10000
 - 9s - loss: 4789.0611 - eucInMM: 9.0346 - eucStdMM: 5.2138 - val_loss: 8936.2066 - val_eucInMM: 12.2057 - val_eucStdMM: 7.5801
Epoch 158/10000
 - 9s - loss: 4751.8247 - eucInMM: 9.0129 - eucStdMM: 5.1659 - val_loss: 8934.3390 - val_eucInMM: 12.2047 - val_eucStdMM: 7.5745
Epoch 159/10000
 - 10s - loss: 4766.7704 - eucInMM: 9.0047 - eucStdMM: 5.2065 - val_loss: 8965.0293 - val_eucInMM: 12.2225 - val_eucStdMM: 7.5968
Epoch 160/10000
 - 9s - loss: 4780.2045 - eucInMM: 9.0422 - eucStdMM: 5.1780 - val_loss: 8899.6415 - val_eucInMM: 12.2067 - val_eucStdMM: 7.5311
Epoch 161/10000
 - 9s - loss: 4793.9805 - eucInMM: 9.0404 - eucStdMM: 5.2127 - val_loss: 8971.1673 - val_eucInMM: 12.2476 - val_eucStdMM: 7.5737

Epoch 00161: ReduceLROnPlateau reducing learning rate to 0.0004304672533180565.
Epoch 162/10000
 - 9s - loss: 4719.0125 - eucIn

Epoch 209/10000
 - 9s - loss: 4588.3230 - eucInMM: 8.8549 - eucStdMM: 5.1011 - val_loss: 8903.0093 - val_eucInMM: 12.1879 - val_eucStdMM: 7.5901
Epoch 210/10000
 - 9s - loss: 4565.3903 - eucInMM: 8.8288 - eucStdMM: 5.0907 - val_loss: 8869.7646 - val_eucInMM: 12.1577 - val_eucStdMM: 7.5794
Epoch 211/10000
 - 8s - loss: 4598.3910 - eucInMM: 8.8510 - eucStdMM: 5.1342 - val_loss: 8908.3215 - val_eucInMM: 12.2076 - val_eucStdMM: 7.5587

Epoch 00211: ReduceLROnPlateau reducing learning rate to 0.00025418660952709616.
Epoch 212/10000
 - 9s - loss: 4575.8144 - eucInMM: 8.8246 - eucStdMM: 5.1267 - val_loss: 9059.2608 - val_eucInMM: 12.3394 - val_eucStdMM: 7.5934
Epoch 213/10000
 - 9s - loss: 4651.1829 - eucInMM: 8.9004 - eucStdMM: 5.1641 - val_loss: 8726.5850 - val_eucInMM: 12.0765 - val_eucStdMM: 7.4800
Epoch 214/10000
 - 10s - loss: 4557.0187 - eucInMM: 8.8162 - eucStdMM: 5.0982 - val_loss: 9064.7197 - val_eucInMM: 12.3317 - val_eucStdMM: 7.6173
Epoch 215/10000
 - 9s - loss: 4632.5587 - eucIn

...

 - 7s - loss: 4267.1239 - eucInMM: 8.5513 - eucStdMM: 4.9758 - val_loss: 8868.1917 - val_eucInMM: 12.1595 - val_eucStdMM: 7.6264
Message did not send. Error: Timed out.
Epoch 3529/10000
 - 7s - loss: 4282.5808 - eucInMM: 8.5559 - eucStdMM: 5.0046 - val_loss: 8834.8733 - val_eucInMM: 12.1355 - val_eucStdMM: 7.6123
Epoch 3530/10000
 - 6s - loss: 4277.6576 - eucInMM: 8.5598 - eucStdMM: 4.9918 - val_loss: 8862.4804 - val_eucInMM: 12.1554 - val_eucStdMM: 7.6245
Epoch 3531/10000
 - 7s - loss: 4269.1754 - eucInMM: 8.5280 - eucStdMM: 5.0143 - val_loss: 8843.3837 - val_eucInMM: 12.1393 - val_eucStdMM: 7.6178
Epoch 3532/10000
 - 7s - loss: 4283.5182 - eucInMM: 8.5555 - eucStdMM: 5.0031 - val_loss: 8851.3189 - val_eucInMM: 12.1469 - val_eucStdMM: 7.6208
Epoch 3533/10000
 - 7s - loss: 4282.7322 - eucInMM: 8.5641 - eucStdMM: 4.9923 - val_loss: 8844.7570 - val_eucInMM: 12.1407 - val_eucStdMM: 7.6174
Message did not send. Error: Timed out.
Epoch 3534/10000
 - 7s - loss: 4260.9731 - eucInMM: 8.5430 - 

Epoch 3578/10000
 - 8s - loss: 4301.5052 - eucInMM: 8.5799 - eucStdMM: 5.0106 - val_loss: 8867.1565 - val_eucInMM: 12.1582 - val_eucStdMM: 7.6256
Message did not send. Error: Timed out.
Epoch 3579/10000
 - 7s - loss: 4250.6169 - eucInMM: 8.5222 - eucStdMM: 4.9910 - val_loss: 8873.9274 - val_eucInMM: 12.1595 - val_eucStdMM: 7.6332
Message did not send. Error: Timed out.
Epoch 3580/10000
 - 7s - loss: 4283.7914 - eucInMM: 8.5678 - eucStdMM: 4.9897 - val_loss: 8848.8445 - val_eucInMM: 12.1408 - val_eucStdMM: 7.6230
Message did not send. Error: Timed out.
Epoch 3581/10000
 - 8s - loss: 4250.0350 - eucInMM: 8.5368 - eucStdMM: 4.9647 - val_loss: 8825.9219 - val_eucInMM: 12.1263 - val_eucStdMM: 7.6119
Epoch 3582/10000
 - 8s - loss: 4243.7767 - eucInMM: 8.5247 - eucStdMM: 4.9640 - val_loss: 8858.9723 - val_eucInMM: 12.1538 - val_eucStdMM: 7.6216
Epoch 3583/10000
 - 8s - loss: 4250.8167 - eucInMM: 8.5384 - eucStdMM: 4.9645 - val_loss: 8849.7805 - val_eucInMM: 12.1447 - val_eucStdMM: 7.6193
Epoc

Epoch 3629/10000
 - 7s - loss: 4234.1847 - eucInMM: 8.5192 - eucStdMM: 4.9527 - val_loss: 8859.5571 - val_eucInMM: 12.1548 - val_eucStdMM: 7.6189
Epoch 3630/10000
 - 7s - loss: 4278.0804 - eucInMM: 8.5444 - eucStdMM: 5.0230 - val_loss: 8847.3697 - val_eucInMM: 12.1481 - val_eucStdMM: 7.6086
Message did not send. Error: Timed out.
Epoch 3631/10000
 - 7s - loss: 4254.6756 - eucInMM: 8.5290 - eucStdMM: 4.9832 - val_loss: 8845.3700 - val_eucInMM: 12.1429 - val_eucStdMM: 7.6140
Message did not send. Error: Timed out.
Epoch 3632/10000
 - 7s - loss: 4265.0518 - eucInMM: 8.5506 - eucStdMM: 4.9746 - val_loss: 8867.6845 - val_eucInMM: 12.1577 - val_eucStdMM: 7.6268
Epoch 3633/10000
 - 8s - loss: 4257.9183 - eucInMM: 8.5482 - eucStdMM: 4.9636 - val_loss: 8864.1961 - val_eucInMM: 12.1543 - val_eucStdMM: 7.6265
Epoch 3634/10000
 - 8s - loss: 4288.9510 - eucInMM: 8.5565 - eucStdMM: 5.0199 - val_loss: 8877.3475 - val_eucInMM: 12.1649 - val_eucStdMM: 7.6314
Epoch 3635/10000
 - 8s - loss: 4244.0643 - e

Epoch 3680/10000
 - 9s - loss: 4274.0778 - eucInMM: 8.5451 - eucStdMM: 5.0095 - val_loss: 8882.0913 - val_eucInMM: 12.1720 - val_eucStdMM: 7.6299
Epoch 3681/10000
 - 8s - loss: 4248.9554 - eucInMM: 8.5351 - eucStdMM: 4.9640 - val_loss: 8875.3880 - val_eucInMM: 12.1617 - val_eucStdMM: 7.6352
Message did not send. Error: Timed out.
Epoch 3682/10000
 - 7s - loss: 4215.1235 - eucInMM: 8.4936 - eucStdMM: 4.9571 - val_loss: 8853.0757 - val_eucInMM: 12.1443 - val_eucStdMM: 7.6258
Message did not send. Error: Timed out.
Epoch 3683/10000
 - 7s - loss: 4237.9846 - eucInMM: 8.5313 - eucStdMM: 4.9400 - val_loss: 8863.3882 - val_eucInMM: 12.1562 - val_eucStdMM: 7.6254
Epoch 3684/10000
 - 8s - loss: 4304.7921 - eucInMM: 8.5738 - eucStdMM: 5.0249 - val_loss: 8879.7626 - val_eucInMM: 12.1671 - val_eucStdMM: 7.6321
Epoch 3685/10000
 - 8s - loss: 4289.0351 - eucInMM: 8.5619 - eucStdMM: 5.0122 - val_loss: 8872.6679 - val_eucInMM: 12.1629 - val_eucStdMM: 7.6290
Epoch 3686/10000
 - 8s - loss: 4234.3075 - e

Epoch 3732/10000
 - 8s - loss: 4278.8269 - eucInMM: 8.5571 - eucStdMM: 4.9960 - val_loss: 8869.7963 - val_eucInMM: 12.1616 - val_eucStdMM: 7.6264
Epoch 3733/10000
 - 8s - loss: 4266.7880 - eucInMM: 8.5427 - eucStdMM: 4.9938 - val_loss: 8864.7197 - val_eucInMM: 12.1567 - val_eucStdMM: 7.6257
Message did not send. Error: Timed out.
Epoch 3734/10000
 - 7s - loss: 4240.2049 - eucInMM: 8.5210 - eucStdMM: 4.9657 - val_loss: 8865.1930 - val_eucInMM: 12.1594 - val_eucStdMM: 7.6243
Message did not send. Error: Timed out.
Epoch 3735/10000
 - 7s - loss: 4227.9357 - eucInMM: 8.5065 - eucStdMM: 4.9621 - val_loss: 8882.2331 - val_eucInMM: 12.1687 - val_eucStdMM: 7.6342
Epoch 3736/10000
 - 8s - loss: 4292.4535 - eucInMM: 8.5654 - eucStdMM: 5.0178 - val_loss: 8852.3621 - val_eucInMM: 12.1484 - val_eucStdMM: 7.6203
Epoch 3737/10000
 - 8s - loss: 4256.4809 - eucInMM: 8.5247 - eucStdMM: 4.9980 - val_loss: 8846.9898 - val_eucInMM: 12.1430 - val_eucStdMM: 7.6192
Epoch 3738/10000
 - 8s - loss: 4255.4267 - e